In [2]:
import sys
sys.path.append(r"E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\ML Lab Spring 25\timeseires")

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback

In [4]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [5]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [6]:
model1 = create_rnn()
model1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 21)]          0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 24, 8)             240       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 20)                580       
_________________________________________________________________
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 841
Trainable params: 841
Non-trainable params: 0
_________________________________________________________________


In [7]:
tensorflow.keras.utils.plot_model(model1 )

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [8]:
checkpoints = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [10]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [11]:
path_dataset =r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\FYP DETAILS\dada sets\AEP\AEP'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [12]:
time_steps=24
num_features=21

In [13]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 3.18430495262146 sec


In [15]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/5
2652/2653 [============================>.] - ETA: 0s - loss: 0.0223 - mae: 0.0223 - mape: 74.5275
Epoch 00001: val_loss improved from inf to 0.01600, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E1-cp-0001-loss0.02.h5
2653/2653 [==============================] - 91s 34ms/step - loss: 0.0223 - mae: 0.0223 - mape: 74.5123 - val_loss: 0.0160 - val_mae: 0.0160 - val_mape: 9.0211
Epoch 2/5
2651/2653 [============================>.] - ETA: 0s - loss: 0.0152 - mae: 0.0152 - mape: 229.1734 ETA: 1s - loss: 0.0152 
Epoch 00002: val_loss improved from 0.01600 to 0.01425, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E1-cp-0002-loss0.01.h5
2653/2653 [==============================] - 86s 32ms/step - loss: 0.0152 - mae: 0.0152 - mape: 229.0405 - val_loss: 0.0143 - val_mae: 0.0143 - val_mape: 6.6053
Epoch 3/5
2653/2653 [==============================] - ETA: 0s - loss: 0.0137 - mae: 0.0137 - mape: 107.0492 ETA: 0s - loss: 0.0

In [16]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E1-cp-0004-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 185.15
Median Absolute Error (MedAE): 157.21
Mean Squared Error (MSE): 54721.94
Root Mean Squared Error (RMSE): 233.93
Mean Absolute Percentage Error (MAPE): 1.28 %
Median Absolute Percentage Error (MDAPE): 1.08 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [17]:
checkpoints = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E1-cp-0004-loss0.01.h5'
start_epoch= 7

In [18]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E1-cp-0004-loss0.01.h5...


[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [19]:
epochs = 5
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/5
2653/2653 [==============================] - ETA: 0s - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 197.4838
Epoch 00001: val_loss improved from inf to 0.01016, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 113s 43ms/step - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 197.4838 - val_loss: 0.0102 - val_mean_absolute_error: 0.0102 - val_mean_absolute_percentage_error: 4.7394
Epoch 2/5
2652/2653 [============================>.] - ETA: 0s - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 248.9462
Epoch 00002: val_loss improved from 0.01016 to 0.01001, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 130s 49ms/step - loss: 0.0108 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 248

In [20]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab9\E2-cp-0005-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 147.75
Median Absolute Error (MedAE): 116.44
Mean Squared Error (MSE): 38308.64
Root Mean Squared Error (RMSE): 195.73
Mean Absolute Percentage Error (MAPE): 1.02 %
Median Absolute Percentage Error (MDAPE): 0.8 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
